### Суть проекта:
#### Написать программу, которая собирает динамически загружаемые товары и складывает данные в БД с помощью Selenium.
#### (сайт сети "Пятерочка" и раздел "Спецпредложения" - 'https://5ka.ru/special_offers'). 

In [1]:
! pip install selenium

In [2]:
! pip install pymongo

In [3]:
# Подключение необходимых модулей и библиотек
from pymongo import MongoClient

from selenium.webdriver.chrome.service import Service
from selenium import webdriver

from selenium.webdriver.chrome.options import Options


from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait

from time import sleep
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from selenium.common.exceptions import NoSuchElementException


from bs4 import BeautifulSoup as bs
import requests
import json

import pandas as pd
from pprint import pprint
from lxml import html

from tqdm import tqdm


In [4]:

s = Service('C:/Users/user/Desktop/chromedriver.exe')
browser = webdriver.Chrome(service=s)


In [5]:
# Раздел "Спецпредложения" сайта сети "Пятерочка" 
url = 'https://5ka.ru/special_offers'

In [6]:
browser.get(url)
#Увеличиваем окно на весь экран
browser.maximize_window()

In [7]:
browser.implicitly_wait(15)

In [8]:
# Убираем кнопку местоположения
button = browser.find_element(By.XPATH, '//*[@id="__layout"]/main/div[1]/div/div[1]/div/div/div[1]/div/div/div/div/div/div[1]/div[2]/button[1]')
button.click()

In [9]:
# Убираем кнопку cookies
button = browser.find_element(By.XPATH, '//*[@id="__layout"]/main/aside/div/div/div/button/div/span')
button.click()

In [10]:
# Организуем нажатие кнопки "Показать еще"
while True:
    wait = WebDriverWait(browser, 15)
    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@class="add-more-btn"]')))
       
        button.click()
    except TimeoutException:
        print('Кнопку найти не удалось')
        break


Кнопку найти не удалось


In [11]:
# Товарные карточки
goods = browser.find_elements(By.XPATH, '//*[@class="product-card item"]')

In [12]:
# Бегло проверяем работоспособность
goods


[<selenium.webdriver.remote.webelement.WebElement (session="4cd1d669702668862ef10bc9cf0bd4b4", element="ee761c84-e2a8-4523-878d-07126948ea73")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4cd1d669702668862ef10bc9cf0bd4b4", element="4d698b37-769f-4cb7-a5f4-874f6b34a468")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4cd1d669702668862ef10bc9cf0bd4b4", element="95c2f627-78e8-414c-9421-3158bb8959e4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4cd1d669702668862ef10bc9cf0bd4b4", element="12e9a0cb-b0a3-40ea-beb6-af6ba3c455bf")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4cd1d669702668862ef10bc9cf0bd4b4", element="354084ce-24d8-448c-a2a6-b174b1322b17")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4cd1d669702668862ef10bc9cf0bd4b4", element="e4d6c214-1f34-4866-a8ae-edff1406612a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4cd1d669702668862ef10bc9cf0bd4b4", element="1b02eede-24db-4a59-a776-a6

In [13]:
# Бегло проверяем работоспособность и правильность выборки товаров
goods[2].find_element(By.XPATH, './/div[@class = "item-name"]').text

'Филе-кусочки Сельди По-Ца...'

In [14]:
goods[0].find_element(By.XPATH, './/div[@class = "discount-hint hint"]').text

'-31 %'

In [19]:
print(goods[0].find_element(By.XPATH, './/span[@class="price-regular"]').text) 

142 .99


In [21]:
# Количество записей
len(goods)

1171

In [17]:
# Организуем хранение полученных данных в Json-файл
goods_list = []
for i in goods:
            goods_list.append(
                {
         # Для каждого товара определяем необходимые параметры                  
                    'name': i.find_element(By.XPATH, './/div[@class = "item-name"]').text, # Название товара
                    'price': i.find_element(By.XPATH, './/span[@class="price-regular"]').text, # Цена
                    'discont': i.find_element(By.XPATH, './/div[@class = "discount-hint hint"]').text, # Скидка
                    
                                       
                }
            )

In [18]:
# Проверяем правильность
goods_list

[{'name': 'Сельдь фил.кус.в майон.пл...',
  'price': '142 .99',
  'discont': '-31 %'},
 {'name': 'Голень цыпленка, охлажден...',
  'price': '299 .99',
  'discont': '-34 %'},
 {'name': 'Филе-кусочки Сельди По-Ца...',
  'price': '189 .99',
  'discont': '-22 %'},
 {'name': 'Горбуша вяленая, соломка,...',
  'price': '89 .99',
  'discont': '-28 %'},
 {'name': 'Fish House Крабовые палоч...',
  'price': '149 .99',
  'discont': '-24 %'},
 {'name': 'Fish House Путассу сушено...',
  'price': '99 .99',
  'discont': '-36 %'},
 {'name': 'Рыбное ассорти Золотая Fi...',
  'price': '149 .99',
  'discont': '-17 %'},
 {'name': 'Скумбрия кусочки холодног...',
  'price': '219 .99',
  'discont': '-32 %'},
 {'name': 'Фарш говяжий, Мираторг, 4...',
  'price': '294 .99',
  'discont': '-43 %'},
 {'name': 'Коктейль из морепрод.в ра...',
  'price': '199 .99',
  'discont': '-31 %'},
 {'name': 'Мидии Fish House, в рассо...',
  'price': '169 .99',
  'discont': '-18 %'},
 {'name': 'Колбаски Чевапчичи из гов...',
  '

In [24]:
# Для наглядности заносим данные в Pandas
goods_result = pd.DataFrame(goods_list)
goods_result

,name,price,discont
0,Сельдь фил.кус.в майон.пл...,142 .99,-31 %
1,"Голень цыпленка, охлажден...",299 .99,-34 %
2,Филе-кусочки Сельди По-Ца...,189 .99,-22 %
3,"Горбуша вяленая, соломка,...",89 .99,-28 %
4,Fish House Крабовые палоч...,149 .99,-24 %
...,...,...,...
1166,"Конфеты Тирамиссо, Слудан...",424 .99,-30 %
1167,"Конфеты Райские облака, с...",538 .99,-13 %
1168,"Конфеты шоколадные, Золот...",579 .99,-32 %
1169,MIX-BAR Гренки рж/пш с че...,74 .99,-14 %


In [25]:
# Экспортируем DataFrame в файл CSV
goods_result.to_csv (r' goods_5ka.csv', encoding='utf-16', index= False )

In [ ]:
# Подготовка БД для приема данных

client = MongoClient('127.0.0.1', 27017)
db = client['goods_5ka']
actions_db = db.goods_5ka


In [118]:
# Заносим данные в базу в коллекцию actions_db
db.actions_db.insert_many(goods_list)

In [120]:
# Проверяем наличие заполнения данных
db.actions_db.find_one()

{'_id': ObjectId('64063cd4470eca482ea810b1'),
 'name': 'Аджика Мистраль Амца, 200...',
 'price': '249 .99',
 'discont': '-25 %'}